In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
import git

# get root of git repo
prefix = git.Repo('.', search_parent_directories=True).working_tree_dir

# project settings
project_uri = prefix+'/mlprojects/sklearn-diabetes'

# azure ml settings
experiment_name = 'sklearn-diabetes-mlproject-example'
compute_target = 'cpu-cluster'

## Setup mlflow

In [ ]:
import mlflow 

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)

In [ ]:
backend_config = {'COMPUTE': compute_target}

## Train model

In [ ]:
run = mlflow.projects.run(uri=project_uri,
                          parameters={"alpha":0.3},
                          backend='azureml',
                          backend_config=backend_config)

In [ ]:
run

## Deploy model

In [ ]:
from azureml.core.webservice import AksWebservice

aks_config = AksWebservice.deploy_configuration(compute_target_name='aks-cpu-deploy',
                                                cpu_cores=2,
                                                memory_gb=5,
                                                tags={'data': 'diabetes', 'method': 'sklearn'}, 
                                                description='Predict using webservice')

In [ ]:
import mlflow.azureml

webservice, azure_model = mlflow.azureml.deploy(model_uri=f'runs:/{run.run_id}/model',
                                                workspace=ws,
                                                deployment_config=aks_config,
                                                service_name='sklearn-diabetes',
                                                model_name='sklearn-diabetes-example')

## Test webservice

In [ ]:
from sklearn import datasets
diabetes = datasets.load_diabetes()

import pandas as pd
import numpy as np

X = diabetes.data
y = diabetes.target
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
sample = data.drop(["progression"], axis=1).iloc[[0]]
                                                 
query_input = sample.to_json(orient='split')
query_input = eval(query_input)
query_input.pop('index', None)

In [ ]:
import requests
import json

headers = {'Content-Type':'application/json'}

response = requests.post(url=webservice.scoring_uri, data=json.dumps(query_input),headers=headers)
print(response.text)

## Clean up

In [ ]:
webservice.delete()